<style type="text/css" media="screen">
a:link { color:#FFFFFF; text-decoration: none; }
a:visited { color:#FFFFFF; text-decoration: none; }
a:hover { color:#FFFFFF; text-decoration: none; }
a:active { color:#FFFFFF; text-decoration: underline; }
</style>

# <a id="restore" style="text-decoration:none;">HPE CSI Driver Data Fabric for Kubernetes</a>
![](imgs/hpedevlogo.svg)
<img src="imgs/mapr-ar21.gif" alt="csi logo" style="float: right;">
<img src="imgs/csilogo1.png" alt="csi logo" style="float: right;">
## How to deploy SQL Server 2019 with an Operator on Kubernetes Cluster

Emmanuel COLUSSI – WW HPE Customer Innovation Center
<br>
<br>
<br>
<br>

## Deploying SQL Server workload
### Restore Database
</br>
<i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;STEP 1: Get POD Name :</br></br>

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
    <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the commands in the next Cells with button <i class="fas fa-play"></i> : </p>
    <ul>
        <li> Get POD Name</li>
        <li> Set variable POD_NAME : with our POD Name value</li>
    </ul>    
 
</div> 


In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl get pods -n mapr-$JUPYTERHUB_USER|grep mssql-deployment-$JUPYTERHUB_USER|awk '{print$1}'"

In [ ]:
export POD_NAME="mssql-deployment-student4-746c69c77b-7psn4"

</br>
<i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;STEP 2: create a backup folder

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
    <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the command in the next Cell with button <i class="fas fa-play"></i> : </p>
</div> 

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl -n mapr-$JUPYTERHUB_USER exec -it $POD_NAME -- mkdir /var/opt/mssql/backup"

</br>
<i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;STEP 3: Copy data backup files

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
    <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the command in the next Cells with button <i class="fas fa-play"></i> : </p>
</div> 

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl cp /home/mssql/db/WideWorldImporters-Full.bak $POD_NAME:/var/opt/mssql/backup -n mapr-$JUPYTERHUB_USER"

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl cp /home/mssql/db/restorewide.sql $POD_NAME:/var/opt/mssql/backup -n mapr-$JUPYTERHUB_USER"

<i class="fas fa-2x fa-info-circle" style="color:#FFAD33;"></i>
> check if files are copied.we must have 2 files: ***restorewide.sql*** and ***WideWorldImporters-Full.bak*** &nbsp;&nbsp;<i class="fas fa-check-circle" style="color:#FFAD33;"></i>

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
    <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the command in the next Cell with button <i class="fas fa-play"></i> : </p>
</div> 

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl -n mapr-$JUPYTERHUB_USER exec -it $POD_NAME -- ls /var/opt/mssql/backup "

</br>
<i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;STEP 4: Now now we can restore our Database&nbsp;&nbsp;<i class="fas fa-2x fa-database" style="color:#FFAD33;"></i>

Get NodePort to used by the mssql-service

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
    <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the commands in the next Cells with button <i class="fas fa-play"></i> : </p>
    <ul>
        <li> Get NodePort : TCP port binding SQL Server service</li>
        <li> Set varibale HOST_NAME : IP_NODE_KUBERNETES,TPC_PORT : Just change the port</li>
        <li> Run restore database </li>
    </ul>    
 
</div> 

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "sql-mapr/get-nodeport-services.sh $JUPYTERHUB_USER"

In [ ]:
export HOST_NAME="arvine,30351"

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "/opt/mssql-tools/bin/sqlcmd -U sa -P HPeinvent@ -S $HOST_NAME -i sql-mapr/restorewide.sql"

<p><i class="fas fa-2x fa-smile" style="color:#FFAD33;"></i>&nbsp;&nbsp;After a few seconds the database is restored.<img src="imgs/cofee1.png" alt="csi logo" width="200px" height="181px" style="float: right;">
</br>

</br>
<i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;STEP 5: Verify the restored database&nbsp;&nbsp;<i class="fas fa-check-circle" style="color:#FFAD33;"></i>


<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
    <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the command in the next Cell with button <i class="fas fa-play"></i> : </p>
    <p><i class="fas fa-list-ol"></i>&nbsp;&nbsp;The query to display a list of database names in your container</p>
</div> 

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "/opt/mssql-tools/bin/sqlcmd -U sa -P HPeinvent@ -S $HOST_NAME -Q 'SELECT Name FROM sys.Databases'"

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
    <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the command in the next Cell with button <i class="fas fa-play"></i> : </p>
    <p><i class="fas fa-list-ol"></i>&nbsp;&nbsp;Run a query to view the top 10 items in the <b>Warehouse.StockItems tabler</b></p>
</div> 

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "/opt/mssql-tools/bin/sqlcmd -U sa -P HPeinvent@ -S $HOST_NAME -Q 'SELECT TOP 10 StockItemID, StockItemName FROM WideWorldImporters.Warehouse.StockItems ORDER BY StockItemID'"

</br></br>
<div class="alert" style="background-color:#FFAD33;color:#fff;width:100%" role="alert">
  <p style="font-size:20px;display: block;text-align:center;margin-left: auto;margin-right: auto;"><i class="fas fa-4x fa-smile"></i>&nbsp;&nbsp;Congratulations on completing the module&nbsp;&nbsp;<i class="fas fa-4x fa-praying-hands"></i></p>
</div>     

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#FFAD33;"></i>&nbsp;&nbsp;Next Steps

Use the persisted data


 <a href="HPEMAPR-SQL-DB1.ipynb#DB1" target="New" title="Next: Use the persisted data"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
 
 <a href="HPEMAPR-SQL.ipynb#sqld" target="New" title="Back: Kubernetes Deployments"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>

<p style="display: block;margin-left: auto;margin-right: auto;width:10%;" ><a href="#restore" title="TOP"><i class="fas fa-2x fa-chevron-circle-up" style="color:#FFAD33;"></i></a></p>